In [1]:
import sqlite3

# Create the SQLite database
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

# Create the sales table
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY,
    product TEXT,
    region TEXT,
    sales INTEGER
)
""")

# Insert sample data
sample_data = [
    ("Product X", "Region A", 100),
    ("Product X", "Region B", 200),
    ("Product Y", "Region A", 150),
    ("Product Y", "Region B", 300)
]
cursor.executemany("INSERT INTO sales (product, region, sales) VALUES (?, ?, ?)", sample_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database and table created successfully!")

Database and table created successfully!


In [2]:
from fpdf import FPDF

# Create a simple PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Customer Feedback", ln=True, align="C")
pdf.ln(10)
pdf.multi_cell(0, 10, txt="Region A: Customers reported durability issues.\n"
                          "Region B: Customers are satisfied with the product.")
pdf.output("feedback.pdf")

print("Test PDF 'feedback.pdf' created successfully!")


Test PDF 'feedback.pdf' created successfully!


In [7]:
import os
from concurrent.futures import ThreadPoolExecutor
import sqlite3
import spacy
from PyPDF2 import PdfReader
from dotenv import load_dotenv
import requests

# Load environment variables
load_dotenv()

# The rest of the imports and classes remain the same until the SummarizationProcessor
# Only showing the modified parts for clarity
# === NLP-Based Query Parser === #
class QueryParser:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")  # Use SpaCy's local model

    def parse(self, query):
        doc = self.nlp(query)
        tasks = [f"Analyze {token.text.lower()}" for token in doc if token.dep_ in ("nsubj", "dobj")]
        return tasks

# === Extended Data Retrieval === #
class SQLDataSource:
    def __init__(self, db_path, query):
        self.db_path = db_path
        self.query = query

    def fetch_data(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(self.query)
        data = cursor.fetchall()
        conn.close()
        return {"sql_data": data}


class DocumentParser:
    def __init__(self, file_path):
        self.file_path = file_path

    def fetch_data(self):
        if self.file_path.endswith(".pdf"):
            text = self._parse_pdf()
        else:
            raise ValueError("Unsupported file format.")
        return {"document_text": text}

    def _parse_pdf(self):
        reader = PdfReader(self.file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text


class APIDataSource:
    def __init__(self, api_url):
        self.api_url = api_url

    def fetch_data(self):
        # Mock API responses
        if "competitors" in self.api_url:
            return {"competitors": "Competitors in Region A offer durable products at competitive prices."}
        elif "market-trends" in self.api_url:
            return {"market_trends": "Growing demand for sustainable and durable products in Region A."}
        else:
            raise ValueError("Invalid API URL")


# === Parallel Execution === #
class ParallelReasoningPipeline:
    def __init__(self):
        self.steps = []

    def add_step(self, description, data_source, processor):
        self.steps.append({"description": description, "data_source": data_source, "processor": processor})

    def run(self):
        results = []

        def process_step(step):
            print(f"\n{step['description']}")
            data = step["data_source"].fetch_data()
            insights = step["processor"].process(data)
            print(f"Insights: {insights}")
            return insights

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_step, step) for step in self.steps]
            for future in futures:
                results.append(future.result())

        return results


# === Enhanced Reasoning Logic === #
class SalesProcessor:
    def process(self, data):
        return f"Sales data: {data['sql_data']}"


class FeedbackProcessor:
    def process(self, data):
        return f"Customer feedback: {data['document_text'][:200]}..."  # Truncate for readability


class CompetitorProcessor:
    def process(self, data):
        return f"Competitor insights: {data['competitors']}"


class MarketTrendsProcessor:
    def process(self, data):
        return f"Market trends: {data['market_trends']}"

class SummarizationProcessor:
    """
    Summarizes insights using the Ollama API.
    """

    def __init__(self, model_name="tinyllama"):
        """
        Initialize the Ollama summarizer.
        :param model_name: Name of the Ollama model to use (default: tinyllama).
        """
        self.model_name = model_name
        self.api_url = "http://localhost:11434/api/generate"

    def process(self, insights):
        """
        Summarizes the combined insights using the Ollama API.
        :param insights: List of insights to summarize.
        :return: Summarized text.
        """
        combined_text = " ".join(insights)
        
        # Prepare the prompt
        prompt = f"""Please summarize the following insights concisely:

{combined_text}

Summary:"""

        # Prepare the request payload
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }

        try:
            # Make request to Ollama API
            response = requests.post(self.api_url, json=payload)
            response.raise_for_status()
            
            # Extract the response
            result = response.json()
            return result.get('response', "Unable to generate summary.")
            
        except requests.exceptions.RequestException as e:
            return f"Error connecting to Ollama: {str(e)}"

def main():
    """
    Entry point for the program. Orchestrates query processing, reasoning steps, and summarization.
    """
    print(" Enhanced Multi-Hop Reasoning Agent \n")

    # Initialize components
    query = "What are the key factors driving the decline in sales for Product X in the last quarter?"
    query_parser = QueryParser()
    tasks = query_parser.parse(query)
    print(f"Parsed Tasks: {tasks}")

    # Initialize the pipeline
    pipeline = ParallelReasoningPipeline()

    # Add reasoning steps
    pipeline.add_step(
        "Analyze sales data",
        SQLDataSource("sales.db", "SELECT * FROM sales WHERE product='Product X'"),
        SalesProcessor(),
    )
    pipeline.add_step(
        "Analyze customer feedback",
        DocumentParser("feedback.pdf"),
        FeedbackProcessor(),
    )
    pipeline.add_step(
        "Analyze competitor data",
        APIDataSource("mock://competitors"),
        CompetitorProcessor(),
    )
    pipeline.add_step(
        "Analyze market trends",
        APIDataSource("mock://market-trends"),
        MarketTrendsProcessor(),
    )

    # Run the pipeline
    insights = pipeline.run()

    # Summarize results using Ollama
    summarizer = SummarizationProcessor(model_name="tinyllama")  # You can change the model as needed
    final_summary = summarizer.process(insights)

    print("\nFinal Summary")
    print(final_summary)

if __name__ == "__main__":
    main()

 Enhanced Multi-Hop Reasoning Agent 

Parsed Tasks: ['Analyze data']

Analyze sales data

Analyze customer feedback

Analyze competitor data
Insights: Competitor insights: Competitors in Region A offer durable products at competitive prices.

Analyze market trends
Insights: Market trends: Growing demand for sustainable and durable products in Region A.
Insights: Sales data: [(1, 'Product X', 'Region A', 100), (2, 'Product X', 'Region B', 200)]
Insights: Customer feedback: Customer Feedback
Region A: Customers reported durability issues.
Region B: Customers are satisfied with the product....

Final Summary
Salé, France - According to market research data, customer feedback reveals that in region A, consumers report issues with durability of the product. However, competitor insight suggests a trend towards growing demand for sustainable and durable products. The insight also highlights the changing demand patterns and increased consumer preferences for quality, reliability, and value-for

In [1]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import sqlite3
from PyPDF2 import PdfReader
from dotenv import load_dotenv
from rich.console import Console
from rich.panel import Panel
import requests

# Initialize console for Rich output
console = Console()

class QueryParser:
    """Parses user queries using LLM to extract actionable tasks"""
    
    def __init__(self):
        self.base_url = "http://localhost:11434/api/chat"

    def parse(self, query):
        """Extracts key analysis targets from user query"""
        prompt = f"""Analyze the following user query and identify the key factors or entities that need to be analyzed. 
        Return your response as a comma-separated list of terms, each term being a noun or noun phrase. 
        Do not include any additional text.

        Query: {query}
        Response:"""
        
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant that extracts key analysis targets from user queries."},
                    {"role": "user", "content": prompt}
                ],
                "stream": False
            }
        )
        
        response_text = response.json()["message"]["content"]
        tasks = [f"Analyze {term.strip()}" for term in response_text.split(",") if term.strip()]
        return tasks


# Data Sources #
class SQLDataSource:
    """Handles retrieving data from an SQLite database."""
    def __init__(self, db_path, query):
        self.db_path = db_path
        self.query = query

    def fetch_data(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(self.query)
        data = cursor.fetchall()
        conn.close()
        return {"sql_data": data}


class DocumentParser:
    """Handles extracting text data from PDF documents."""
    def __init__(self, file_path):
        self.file_path = file_path

    def fetch_data(self):
        if self.file_path.endswith(".pdf"):
            text = self._parse_pdf()
        else:
            raise ValueError("Unsupported file format.")
        return {"document_text": text}

    def _parse_pdf(self):
        reader = PdfReader(self.file_path)
        text = "".join([page.extract_text() for page in reader.pages])
        return text


class APIDataSource:
    """Handles API data retrieval for competitor and market data."""
    def __init__(self, api_url):
        self.api_url = api_url

    def fetch_data(self):
        if "competitors" in self.api_url:
            return {"competitors": "Competitors in Region A offer durable products at competitive prices."}
        elif "market-trends" in self.api_url:
            return {"market_trends": "Growing demand for sustainable and durable products in Region A."}
        else:
            raise ValueError("Invalid API URL")


# Parallel Processing #
class ParallelReasoningPipeline:
    """Manages parallel execution of reasoning tasks with dynamic step numbering."""
    def __init__(self):
        self.steps = []

    def add_step(self, description, data_source, processor, data_type, color):
        """Adds an analysis step to the pipeline without predefined numbering."""
        self.steps.append({
            "description": description,
            "data_source": data_source,
            "processor": processor,
            "data_type": data_type,
            "color": color
        })

    def run(self):
        """Executes all pipeline steps in parallel and numbers them dynamically."""
        results = []
        step_number = 1  # Start numbering dynamically

        with ThreadPoolExecutor() as executor:
            future_to_step = {executor.submit(self.process_step, step): step for step in self.steps}

            for future in as_completed(future_to_step):
                step = future_to_step[future]
                insights = future.result()
                
                # Dynamically number the step
                step_content = (
                    f"[bold]Data Source:[/bold] {step['data_type']}\n"
                    f"[bold]Insights:[/bold] {insights}"
                )
                console.print(Panel(step_content, title=f"[bold]Step {step_number}: {step['description']}[/bold]", border_style=step["color"]))
                
                results.append(insights)
                step_number += 1  # Increment step number for next completed step

        return results

    def process_step(self, step):
        """Fetches data, processes it, and returns insights."""
        data = step["data_source"].fetch_data()
        return step["processor"].process(data)


# Data Processors #
class SalesProcessor:
    """Processes sales data using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        if not data["sql_data"]:
            return "No sales data available."

        # Format data for LLM analysis
        sales_records = "\n".join([str(record) for record in data["sql_data"]])

        # Prompt LLM to analyze sales trends
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following sales data and provide insights:\n\n{sales_records}",
                "system": "You are an expert business analyst.",
                "stream": False
            }
        )
        return response.json()["response"]


class FeedbackProcessor:
    """Processes customer feedback using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        feedback_text = data["document_text"]

        if not feedback_text.strip():
            return "No feedback data available."

        # Prompt LLM to analyze customer sentiment
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following customer feedback and provide insights:\n\n{feedback_text}",
                "system": "You are an expert in customer sentiment analysis.",
                "stream": False
            }
        )
        return response.json()["response"]


class CompetitorProcessor:
    """Processes competitor data using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        competitor_info = data.get("competitors", "")

        if not competitor_info.strip():
            return "No competitor data available."

        # Prompt LLM to analyze competitor strategy
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following competitor data and provide insights:\n\n{competitor_info}",
                "system": "You are an expert in competitor analysis.",
                "stream": False
            }
        )
        return response.json()["response"]


class MarketTrendsProcessor:
    """Processes market trends using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        market_trends_info = data.get("market_trends", "")

        if not market_trends_info.strip():
            return "No market trend data available."

        # Prompt LLM to analyze market trends
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following market trend data and provide insights:\n\n{market_trends_info}",
                "system": "You are an expert market analyst.",
                "stream": False
            }
        )
        return response.json()["response"]


class SummarizationProcessor:
    """Summarizes insights using LLM."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, insights):
        """Generates final summary using LLM."""
        combined_text = " ".join(insights)
        
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": combined_text,
                "system": "Create a concise, actionable summary of the following business insights.",
                "stream": False
            }
        )
        return response.json()["response"]


def main():
    console.print("[bold cyan]\nEnhanced Multi-Hop Reasoning Agent[/bold cyan]\n")

    # Parse the query using LLM
    query = "What are the key factors driving the decline in sales for Product X in the last quarter?"
    query_parser = QueryParser()
    tasks = query_parser.parse(query)
    console.print(Panel(f"Parsed Tasks: {tasks}", title="Query Parsing", border_style="cyan"))

    # Initialize the pipeline
    pipeline = ParallelReasoningPipeline()

    # Add steps to pipeline
    pipeline.add_step(
        "Retrieve and analyze sales data for Product X",
        SQLDataSource("sales.db", "SELECT * FROM sales WHERE product='Product X'"),
        SalesProcessor(),
        "SQL Database", "blue"
    )
    pipeline.add_step(
        "Retrieve and analyze customer feedback for Product X",
        DocumentParser("feedback.pdf"),
        FeedbackProcessor(),
        "Document", "green"
    )
    pipeline.add_step(
        "Retrieve and analyze competitor data",
        APIDataSource("mock://competitors"),
        CompetitorProcessor(),
        "API", "yellow"
    )
    pipeline.add_step(
        "Retrieve and analyze market trends",
        APIDataSource("mock://market-trends"),
        MarketTrendsProcessor(),
        "API", "red"
    )

    # Run the pipeline
    insights = pipeline.run()

    # Summarize results using LLM
    summarizer = SummarizationProcessor()
    final_summary = summarizer.process(insights)

    console.print(Panel(final_summary, title="[bold]Final Summary[/bold]", border_style="magenta"))


if __name__ == "__main__":
    main()

Enhanced Multi-Hop Reasoning Agent

╭───────────────────────────────────────────────── Query Parsing ─────────────────────────────────────────────────╮
│ Parsed Tasks: ['Analyze Key Factors Driving Decline in Sales for Product X in the Last Quarter:\n1.             │
│ Competition: As mentioned by another user query', "Analyze Product X is heavily competitive in its market       │
│ segment. This makes it tough for it to stand out and gain more customers. \n2. Pricing: Product X's pricing     │
│ strategy has not been successful in maintaining a profitable margin for the company. This has caused            │
│ significant losses in sales and profitability. \n3. Supply Chain Management: As per another query", 'Analyze    │
│ due to a lack of proper supply chain management', "Analyze Product X faced supply shortages and higher costs    │
│ that made it difficult for the business to compete with its competitors. \n4. Customer Relationship Management: │
│ The company's customer relationship management strategy has not been effective in keeping customers loyal to    │
│ their brand. This has resulted in decreased customer satisfaction and a loss of sales. \n5. Market Trends:      │
│ Market trends such as the COVID-19 pandemic have impacted Product X's sales in the last quarter", 'Analyze      │
│ causing a decline in overall revenue for the company.']                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────── Step 1: Retrieve and analyze customer feedback for Product X ──────────────────────────╮
│ Data Source: Document                                                                                           │
│ Insights: Regarding region A, based on the given feedback, the customers reported that the product has          │
│ durability issues. This is likely due to the high quality of the material used or poor craftsmanship. The       │
│ customers were not fully satisfied and mentioned that they would like the product to have better durability in  │
│ the future.                                                                                                     │
│                                                                                                                 │
│ Regarding region B, the customers were generally pleased with the product, indicating that it meets their       │
│ expectations. They reported being satisfied with the product's performance and functionality. It is likely due  │
│ to the customer-centric approach taken by the company to produce high-quality products that align with their    │
│ needs. The feedback provided suggests that the product is effective in meeting customers' expectations.         │
│                                                                                                                 │
│ In summary, based on the given feedback, the customers reported several issues with durability but expressed    │
│ satisfaction overall with the product's performance and functionality. This suggests that the company has taken │
│ a customer-centric approach to produce high-quality products that align with their needs, resulting in          │
│ satisfied customers.                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────── Step 2: Retrieve and analyze sales data for Product X ─────────────────────────────╮
│ Data Source: SQL Database                                                                                       │
│ Insights: Regarding the above sales data, we can see that there is a significant difference between the revenue │
│ generated by Product X in Region A compared to Region B. In Region A, the revenue generated was higher than in  │
│ Region B by approximately 50%. This suggests that Product X has a stronghold over the Region A market and is    │
│ more successful there than in Region B.                                                                         │
│                                                                                                                 │
│ This analysis can provide insights into the following:                                                          │
│ - The performance of Product X in different geographic areas, such as Region A or Region B.                     │
│ - The strengths and weaknesses of Product X based on its performance in each region.                            │
│ - The effectiveness of marketing strategies for Product X in different regions to attract a larger customer     │
│ base.                                                                                                           │
│ - The potential opportunities and challenges for expanding into new geographic areas for Product X.             │
│                                                                                                                 │
│ By analyzing the sales data, we can gain insights into how Product X performs across different regions,         │
│ identify any trends or patterns, and determine the best approach for expanding into new markets.                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── Step 3: Retrieve and analyze market trends ───────────────────────────────────╮
│ Data Source: API                                                                                                │
│ Insights: Trench D data from Growiing is as follows:                                                            │
│                                                                                                                 │
│ 1) Growiing has tracked consumer trends in the region of A for sustainability and durability of products over   │
│ time, with a steady increase in demand for these types of products.                                             │
│                                                                                                                 │
│ 2) The rise in demand can be attributed to several factors, including consumers' growing concerns about         │
│ environmental sustainability, social responsibility, and cost-effectiveness of sustainable alternatives.        │
│                                                                                                                 │
│ 3) Consumers in region A are looking for high quality, eco-friendly, and long-lasting products that are made    │
│ from renewable resources, such as wood, recycled plastics, or bamboo. This demand is driving the growth of the  │
│ sustainable and durable product sector.                                                                         │
│                                                                                                                 │
│ 4) Growiing data shows that demand for these products is highest in urban areas, where residents have higher    │
│ disposable income and are more willing to pay a premium for environmentally-friendly alternatives.              │
│                                                                                                                 │
│ 5) Additionally, the rising demand for sustainable and durable products is being driven by a changing consumer  │
│ behavior, with more consumers prioritizing sustainability over convenience in their purchasing decisions.       │
│                                                                                                                 │
│ 6) In conclusion, Growiing data shows that there is a strong correlation between the rise in consumer demand    │
│ for sustainable and durable products in region A, and the increasing popularity of these products among         │
│ consumers. The market trends highlight how consumer preferences are shifting towards environmentally-friendly   │
│ alternatives, which are driving the growth of this sector.                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── Step 4: Retrieve and analyze competitor data ──────────────────────────────────╮
│ Data Source: API                                                                                                │
│ Insights: I do not have access to the specific data you are referring to, but based on general competitor       │
│ analysis techniques, here are some insights based on the provided information:                                  │
│                                                                                                                 │
│ 1. Comparison of Products and Pricing: When analyzing competitors in region a, we would expect them to offer    │
│ durable products that are priced similarly or below the industry average. This could be due to their use of     │
│ cost-effective materials or sourcing strategies. For example, if one company offers tires at a premium price,   │
│ but sources them from a low-cost supplier, this could lead to higher prices for their competitors.              │
│                                                                                                                 │
│ 2. Differences in Target Market: Region a is likely a region where consumers have more spending power than in   │
│ other parts of the country. This could mean that competitors offer differentiation and value propositions       │
│ tailored to target market needs. For example, one company may focus on high-end tires for luxury vehicles while │
│ another competes with lower prices for mass-market vehicles.                                                    │
│                                                                                                                 │
│ 3. Differences in Brand Image: Region a is likely a region where there are established brand names that         │
│ consumers are familiar with. This could mean that competitors offer differentiation and value propositions      │
│ based on the perceived quality or reliability of their brands. For example, one company may offer premium tires │
│ from top-rated suppliers while another competes with lower prices for generic or subpar brands.                 │
│                                                                                                                 │
│ 4. Differences in Products: Region a is likely a region where there are differences in product lines and        │
│ features. This could mean that competitors specialize in different types of products, such as tires for         │
│ off-road vehicles versus regular street tires. For example, one company may focus on high-end off-roading       │
│ vehicles while another competes with lower prices for regular cars.                                             │
│                                                                                                                 │
│ 5. Differences in Target Demographics: Region a is likely a region where there are differences in demographics. │
│ This could mean that competitors offer differentiation and value propositions based on the needs and            │
│ preferences of different types of customers. For example, one company may focus on providing high-end tires for │
│ luxury vehicle owners while another competes with lower prices for regular drivers.                             │
│                                                                                                                 │
│ by analyzing the data provided, it appears that region a is likely a region where there are differences in      │
│ product lines and features, as well as target demographics, which can help to differentiate and promote their   │
│ offerings.                                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Final Summary ─────────────────────────────────────────────────╮
│ Based on the feedback from the given sales data, here are some insights:                                        │
│                                                                                                                 │
│ 1) Region A is likely a region where sustainability and durability of products are key trends. The rise in      │
│ demand for these types of products can be attributed to several factors, including consumer concerns about      │
│ environmental sustainability, social responsibility, and cost-effectiveness of sustainable alternatives.        │
│ Consumer preference for these products is shifting towards environmentally-friendly alternatives due to         │
│ changing consumer behavior.                                                                                     │
│                                                                                                                 │
│ 2) Growiing data shows that consumers in region A are looking for high quality, ecotoxicology-compliant, and    │
│ long-lasting products made from renewable resources such as wood, recycled plastics, or bamboo. The rise of     │
│ sustainability as a priority among consumers is driving demand for these types of products.                     │
│                                                                                                                 │
│ 3) Growiing data also shows that in region A, higher disposable income and a preference for                     │
│ environmentally-friendly alternatives are driving the growth of the sustainable and durable product sector. As  │
│ more consumers prioritize sustainability over convenience, there is an increasing demand for these types of     │
│ products.                                                                                                       │
│                                                                                                                 │
│ 4) In conclusion, Growiing data highlights the trend towards sustainable and durable products in region A. The  │
│ market trends show that consumer preference for these products are shifting towards environmentally-friendly    │
│ alternatives, driven by changing consumer behavior. Consumers in this region prioritize sustainability over     │
│ convenience, which is driving the growth of this sector.                                                        │
│                                                                                                                 │
│ 5) Comparing product offerings and pricing between competitors in region A, we would expect companies to offer  │
│ durable products that are priced similarly or below the industry average. This could be due to their use of     │
│ cost-effective materials or sourcing strategies. In region A, sustainability and durability are key trends, so  │
│ it is likely that competitors will prioritize these factors in their product offerings.                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯